 # I. Création de la base de donées annotées

## I.1. Capture des photos

In [1]:
import cv2
from cvzone import HandTrackingModule

In [ ]:
#changer le label en fonction du dataset
label = input("Entree le label : ")
cnt_img = 100
detector = HandTrackingModule.HandDetector()

In [ ]:
#Initialisation de la webcam
capture = cv2.VideoCapture(0)

while True:
    #capture d'une image du flux de la webcam
    ret,img = capture.read()
    img_copy = img.copy()
    hands, img = detector.findHands(img)

    if not ret:
        print("Erreur lors de la lecture de img")
        break

    #Affichage de l'image
    cv2.imshow("Image", img)

    key = cv2.waitKey(1)
    # ESC 
    if key%256 == 27:
        print("ESC, fermeture...")
        break
    # ESPACE
    elif key%256 == 32:

        bbox_value = hands[0].get('bbox')

        #Ecrit le roi dans le fichier
        roi = img_copy[bbox_value[1]:bbox_value[1] + bbox_value[3], bbox_value[0]:bbox_value[0] + bbox_value[2]]

        img_name = "image_final/{0}/{0}_{1}.png".format(label,cnt_img)
        cv2.imwrite(img_name, roi)
        print("{} ecrit!".format(img_name))
        cnt_img += 1


capture.release()
cv2.destroyAllWindows()   

## I.2. Gauss

In [2]:
import cv2
import numpy as np
import os

In [3]:

def gaussian_filter(img):
    # Charger l'image
    image = cv2.imread(img)
    # Appliquer le filtre Gaussien
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    # Enregistrer l'image filtrée
    cv2.imwrite(img, blurred)

In [4]:

root_dir = 'image_final'
for sub_dir in os.listdir(root_dir):
    sub_dir_path = os.path.join(root_dir, sub_dir)
    for filename in os.listdir(sub_dir_path):
        img = os.path.join(sub_dir_path, filename)
        gaussian_filter(img)


## I.2. Data Augmentation

In [5]:
import Augmentor
import os

In [6]:
root_dir = 'image_final'

In [7]:
for sub_dir in os.listdir(root_dir):
    sub_dir_path = os.path.join(root_dir, sub_dir)

    p = Augmentor.Pipeline(sub_dir_path)

    p.zoom(probability=0.3,min_factor=0.8,max_factor=1.5)
    p.flip_top_bottom(probability=0.4)
    p.random_brightness(probability=0.3,min_factor=0.3,max_factor=1.2)
    p.random_distortion(probability=1,grid_width=4,grid_height=4,magnitude=8)

    p.sample(500)

    output_dir = os.path.join(sub_dir_path, 'output')
    if os.path.isdir(output_dir):
        for filename in os.listdir(output_dir):
            src = os.path.join(output_dir, filename)
            dst = os.path.join(sub_dir_path, filename)
            os.rename(src, dst)
        os.rmdir(output_dir)

Initialised with 91 image(s) found.
Output directory set to image_final\A\output.

Processing <PIL.Image.Image image mode=RGB size=137x189 at 0x1DA5D79BA30>: 100%|██████████| 500/500 [00:03<00:00, 151.80 Samples/s]


Initialised with 79 image(s) found.
Output directory set to image_final\B\output.

Processing <PIL.Image.Image image mode=RGB size=85x200 at 0x1DA5D7B7FA0>: 100%|██████████| 500/500 [00:02<00:00, 190.07 Samples/s] 


Initialised with 84 image(s) found.
Output directory set to image_final\C\output.

...

Processing <PIL.Image.Image image mode=RGB size=47x94 at 0x1DA5D77CD90>: 100%|██████████| 500/500 [00:02<00:00, 186.40 Samples/s]  


Initialised with 71 image(s) found.
Output directory set to image_final\V\output.

Processing <PIL.Image.Image image mode=RGB size=99x207 at 0x1DA5E8145E0>: 100%|██████████| 500/500 [00:02<00:00, 190.39 Samples/s] 


Initialised with 71 image(s) found.
Output directory set to image_final\W\output.

Processing <PIL.Image.Image image mode=RGB size=72x89 at 0x1DA5D7626E0>: 100%|██████████| 500/500 [00:01<00:00, 263.21 Samples/s]  


# II. Création et entrainement d'un réseau de neurones

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import cv2
import pydot

In [9]:
data_dir = "image_final/"
labels = sorted(os.listdir(data_dir))

X = []
y = []
for idx, label in enumerate(labels):
  for file in os.listdir(data_dir + '/'+label):
    filepath = data_dir +'/'+ label + "/" + file
    img = cv2.resize(cv2.imread(filepath),(50,50))
    X.append(img)
    y.append(idx)
X = np.asarray(X)
y = np.asarray(y)
print(X.shape, y.shape)

(5742, 50, 50, 3) (5742,)


In [10]:
#spliting dataset into 80% train, 10% validation and 10% test data
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.8)
X_test, X_eval, Y_test, Y_eval = train_test_split(X_test, Y_test, test_size = 0.5)


In [11]:
# converting Y_tes and Y_train to One hot vectors using to_categorical
# example of one hot => '1' is represented as [0. 1. 0. . . . . 0.]
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
Y_eval = to_categorical(Y_eval)
X_train = X_train / 255.
X_test = X_test/ 255.
X_eval = X_eval/ 255.

In [12]:
# building our model
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu', input_shape=(50,50,3)),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 16)        2320      
                                                                 
 conv2d_2 (Conv2D)           (None, 44, 44, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 16)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 32)        4640      
                                                                 
 conv2d_4 (Conv2D)           (None, 18, 18, 32)        9248      
                                                        

In [13]:
#compiling the model
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer,
               loss = 'categorical_crossentropy',
               metrics=['accuracy'],)

In [14]:
#start training(fitting) the data
history = model.fit(X_train, Y_train, epochs=36, verbose=1,
                validation_data=(X_eval, Y_eval))

Epoch 1/36
36/36 [==============================] - 6s 112ms/step - loss: 2.2931 - accuracy: 0.1080 - val_loss: 2.2220 - val_accuracy: 0.1280
Epoch 2/36
36/36 [==============================] - 4s 105ms/step - loss: 1.9705 - accuracy: 0.2805 - val_loss: 1.6869 - val_accuracy: 0.3944
Epoch 3/36
36/36 [==============================] - 3s 97ms/step - loss: 1.5137 - accuracy: 0.5009 - val_loss: 1.4456 - val_accuracy: 0.5159
Epoch 4/36
36/36 [==============================] - 4s 112ms/step - loss: 1.1679 - accuracy: 0.6010 - val_loss: 1.1003 - val_accuracy: 0.6291
Epoch 5/36
36/36 [==============================] - 4s 104ms/step - loss: 0.9788 - accuracy: 0.6611 - val_loss: 1.0759 - val_accuracy: 0.6452
Epoch 6/36
36/36 [==============================] - 4s 100ms/step - loss: 0.8171 - accuracy: 0.7361 - val_loss: 0.8969 - val_accuracy: 0.7048
Epoch 7/36
36/36 [==============================] - 4s 100ms/step - loss: 0.6657 - accuracy: 0.7692 - val_loss: 0.7897 - val_accuracy: 0.7431
Epoch 8

In [15]:
#testing
model.evaluate(X_test, Y_test)

72/72 [==============================] - 1s 15ms/step - loss: 1.2901 - accuracy: 0.8698


[1.2901418209075928, 0.8698301911354065]

In [ ]:
model.save("model_save/model_avec_gauss.h5")

# III. Détéction

## III.1. Détéction avec des images

## III.2. Détéction avec un flux vidéo